Tugas 7 - Keyword Extraction Menggunakan Centrality

Nama : Isnita Widyur Rahmah
NIM : 220411100048
Kelas : IF 7A

Link Project : https://github.com/taawdyy/ppw

##Install Pustaka Python

In [1]:
!pip install selenium networkx matplotlib nltk Sastrawi tqdm pandas sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


#Impor Pustaka

In [16]:
import numpy as np
import pandas as pd
import requests
import re
import networkx as nx

from urllib.request import urlopen
from bs4 import BeautifulSoup

from tqdm import tqdm

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Crawling Data Artikel Berita Menggunakan Python
melakukan crawling untuk mengambil judul, isi, tanggal, dan kategori artikel dari URL yang diberikan, dan menyajikan data tersebut dalam bentuk DataFrame untuk kemudahan analisis lebih lanjut.

In [4]:
def crawl_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Memastikan permintaan berhasil
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mengambil judul
        title_element = soup.find('h1', class_='text-cnn_black')
        title = title_element.get_text().strip() if title_element else 'Judul tidak ditemukan'

        # Mengambil Isi
        content_div = soup.find('div', class_='detail-text')
        content = "\n".join([p.get_text().strip() for p in content_div.find_all('p')]) if content_div else 'Isi artikel tidak ditemukan'

        # Mengambil tanggal
        date_div = soup.find('div', class_='text-cnn_grey text-sm mb-4')
        date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'

        # Mengambil kategori
        category_meta = soup.find("meta", attrs={'name': 'dtk:namakanal'})
        category = category_meta['content'].strip() if category_meta and 'content' in category_meta.attrs else 'Kategori tidak ditemukan'

        return {'Judul': title, 'Isi': content, 'Tanggal': date_text, 'Kategori': category}
    except requests.RequestException as e:
        print(f"Error fetching article: {e}")
        return None

article_url = input("Masukkan URL artikel: ")

# Melakukan crawl pada satu artikel
article = crawl_article(article_url)

df = pd.DataFrame([article])  # Membuat DataFrame dari dictionary
df

Masukkan URL artikel: https://www.cnnindonesia.com/ekonomi/20241016193856-85-1156148/bahlil-sebut-54-persen-warga-morowali-kena-asma-imbas-hilirisasi


,Judul,Isi,Tanggal,Kategori
0,Bahlil Sebut 54 Persen Warga Morowali Kena Asm...,Menteri ESDM Bahlil Lahadalia mengungkap 54 pe...,"Rabu, 16 Okt 2024 20:30 WIB",ekonomi


#Transformasi Teks Menjadi Huruf Kecil
mengubah semua teks dalam kolom Isi menjadi huruf kecil dan menyimpannya dalam kolom baru.

In [5]:
# Fungsi untuk mengubah teks menjadi huruf kecil
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

df['lower case'] = df['Isi'].apply(clean_lower)
df['lower case']

,lower case
0,menteri esdm bahlil lahadalia mengungkap 54 pe...


#Pembersihan Teks dari Tanda Baca dan Angka
membersihkan teks dari tanda baca dan angka, serta menghapus karakter yang tidak diinginkan, dan menyimpannya dalam kolom baru.

In [6]:
# Fungsi untuk membersihkan tanda baca dan angka (menghapus koma)
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z .]+')  # Koma dihilangkan dari pengecualian
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df['tanda baca'] = df['lower case'].apply(clean_punct)
df['tanda baca']

,tanda baca
0,menteri esdm bahlil lahadalia mengungkap perse...


#Normalisasi Spasi dalam Teks
melakukan normalisasi spasi dalam teks dengan mengganti beberapa spasi berturut-turut menjadi satu spasi dan menghapus spasi di awal dan akhir.

In [7]:
# Fungsi untuk normalisasi spasi
def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

df['spasi'] = df['tanda baca'].apply(_normalize_whitespace)
df['spasi']

,spasi
0,menteri esdm bahlil lahadalia mengungkap perse...


#Tokenisasi Teks Menjadi Kata
melakukan tokenisasi pada teks yang telah dinormalisasi, memecahnya menjadi daftar kata atau token, dan menyimpannya dalam kolom baru.

In [8]:
# Tokenisasi
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return text

df['token'] = df['spasi'].apply(tokenize_text)
df['token']

,token
0,"[menteri, esdm, bahlil, lahadalia, mengungkap,..."


#Pembersihan Stopwords dari Token
membersihkan daftar token dari kata-kata umum yang tidak memberikan makna penting (stopwords), sehingga hanya kata-kata yang relevan yang tersisa untuk analisis lebih lanjut.

In [9]:
# Pembersihan stopwords setelah tokenisasi
def clean_stopwords(tokens):
    if isinstance(tokens, list):
        stopword = set(stopwords.words('indonesian'))
        filtered_tokens = [word for word in tokens if word not in stopword]
        return filtered_tokens
    return tokens

df['stopwords'] = df['token'].apply(clean_stopwords)
df['stopwords']

,stopwords
0,"[menteri, esdm, bahlil, lahadalia, mengungkap,..."


#Penggabungan Token Menjadi Teks yang Diproses
menggabungkan kembali daftar token yang telah dibersihkan menjadi string utuh, yang kemudian dapat digunakan untuk analisis lebih lanjut.

In [11]:
# Gabungkan kembali token menjadi string
df['processed_text'] = df['stopwords'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else '')

prep_result = df['processed_text'].to_dict()
print(prep_result)

{0: 'menteri esdm bahlil lahadalia mengungkap persen warga sulawesi terjangkit infeksi saluran pernapasan akut ispa imbas kebijakan hilirisasi . berdasarkan kajian pt indonesia morowali industrial park imip pt indonesia weda bay industrial park iwip . informasi disertasi kajian akademiknya . bahlil mengungkap hilirisasi dampak positif perekonomian . dampak kesehatan buruk . advertisement scroll to continue with content kegiatan ekonomi kesehatan buruk . kesehatan ispa sulawesi morowali persen kena asma sidang doktor gedung makara art ui rabu . menurutnya kesehatan muncul imbas kondisi air udara buruk . proses hilirisasi kali morowali memiliki konsep . berbeda weda bay hijau . proses hilirisasi morowali contoh perbaikan . halmahera air morowali ampun . morowali barang . hasil kajian dampak ekonomi kebijakan hilirisasi penduduk lingkar pertambangan . penghasilan tembus rp juta rp juta . karyawan smelter memaksimalkan ruang ekonomi membangun bisnis kos kosan makanan pungkasnya . gambas vi

#Kumpulan Kalimat dan Kata Unik dari Teks Terproses
menyiapkan data untuk analisis lebih lanjut dengan membuat kumpulan kalimat dari teks yang telah diproses dan mengumpulkan kata-kata unik

In [12]:
# Membuat kumpulan kalimat
sentences = []
for p in df['processed_text']:
  sentences.extend(sent_tokenize(p))

# Membuat kumpulan kata unik
vocabulary = set()
for sentence in sentences:
  for word in sentence.split():
    vocabulary.add(word)

print(sentences)
print(vocabulary)

['menteri esdm bahlil lahadalia mengungkap persen warga sulawesi terjangkit infeksi saluran pernapasan akut ispa imbas kebijakan hilirisasi .', 'berdasarkan kajian pt indonesia morowali industrial park imip pt indonesia weda bay industrial park iwip .', 'informasi disertasi kajian akademiknya .', 'bahlil mengungkap hilirisasi dampak positif perekonomian .', 'dampak kesehatan buruk .', 'advertisement scroll to continue with content kegiatan ekonomi kesehatan buruk .', 'kesehatan ispa sulawesi morowali persen kena asma sidang doktor gedung makara art ui rabu .', 'menurutnya kesehatan muncul imbas kondisi air udara buruk .', 'proses hilirisasi kali morowali memiliki konsep .', 'berbeda weda bay hijau .', 'proses hilirisasi morowali contoh perbaikan .', 'halmahera air morowali ampun .', 'morowali barang .', 'hasil kajian dampak ekonomi kebijakan hilirisasi penduduk lingkar pertambangan .', 'penghasilan tembus rp juta rp juta .', 'karyawan smelter memaksimalkan ruang ekonomi membangun bisni

#Fungsi untuk Menghitung Frekuensi Kata dalam Kalimat
menghasilkan representasi frekuensi kata dalam bentuk DataFrame, yang berguna untuk analisis teks lebih lanjut, seperti pembelajaran mesin atau analisis statistik.

In [13]:
def vektor_kata(data):
	vektor_kata = pd.DataFrame(0, index=range(len(data)), columns=vocabulary)

	for i, sent in enumerate(data):
		# Tokenisasi kalimat menjadi kata-kata
		kata_kalimat = word_tokenize(sent)

		# Hitung frekuensi setiap kata dalam kalimat
		for word in kata_kalimat:
			if word in vocabulary:
				vektor_kata.at[i, word] += 1

	return vektor_kata

#Membuat DataFrame untuk Frekuensi Kata dalam Kalimat
mengonversi kumpulan kalimat menjadi representasi vektor frekuensi kata dalam bentuk DataFrame.

In [14]:
pd.set_option('future.no_silent_downcasting', True)
df = pd.DataFrame(columns=list(vocabulary), index=sentences)

# Mengisi nilai kosong dengan 0
df = df.fillna(0)  # Fill with 0s

# Menghitung nilai setiap kata pada kalimat
for i, sentence in enumerate(sentences):
  for word in sentence.split():
    df.loc[sentence, word] += 1

# Melihat isi Dataframe
df

,kegiatan,berbeda,scroll,hasil,berdasarkan,ispa,asma,cnn,with,rabu,...,kondisi,ui,iwip,pertambangan,ekonomi,park,to,menurutnya,akut,akademiknya
menteri esdm bahlil lahadalia mengungkap persen warga sulawesi terjangkit infeksi saluran pernapasan akut ispa imbas kebijakan hilirisasi .,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
berdasarkan kajian pt indonesia morowali industrial park imip pt indonesia weda bay industrial park iwip .,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,2,0,0,0,0
informasi disertasi kajian akademiknya .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
bahlil mengungkap hilirisasi dampak positif perekonomian .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dampak kesehatan buruk .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
advertisement scroll to continue with content kegiatan ekonomi kesehatan buruk .,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
kesehatan ispa sulawesi morowali persen kena asma sidang doktor gedung makara art ui rabu .,0,0,0,0,0,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
menurutnya kesehatan muncul imbas kondisi air udara buruk .,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
proses hilirisasi kali morowali memiliki konsep .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
berbeda weda bay hijau .,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Menghitung dan Menampilkan Frekuensi Kata Teratas
menghitung dan menampilkan kata-kata dengan frekuensi tertinggi dari DataFrame, memberikan wawasan tentang kata-kata yang paling umum muncul dalam kumpulan kalimat.

In [15]:
# Menghitung jumlah kemunculan pada tiap kata
word_frequencies = df.sum(axis=0)

# Mengurutkan berdasarkan jumlah kemunculan kata
sorted_word_frequencies = word_frequencies.sort_values(ascending=False)

# Menampilkan kata teratas
keyword = 3

for i, (word, freq) in enumerate(sorted_word_frequencies.items()):
    if i < keyword:
        print(f"Rank {i+1}: {word} (Frequency: {freq})")

Rank 1: . (Frequency: 16)
Rank 2: morowali (Frequency: 6)
Rank 3: hilirisasi (Frequency: 5)
